In [ ]:
#Imports
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import xarray as xr
import folium
import numpy as np
import pandas as pd
import datetime as dt

from eodag import EODataAccessGateway
from eodag import setup_logging

from rasterio.crs import CRS
from rioxarray.merge import merge_arrays

from dotenv import dotenv_values

import geopandas as gpd
from sklearn import svm, tree
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
import matplotlib.colors as colors


# from shapely.geometry import MultiPolygon
# from shapely.geometry import shape


# Setup Verbose Values:
# 0: no logging and no progress bar
# 1: no logging but progress bars displayed
# 2: log at the INFO level
# 3: log at the DEBUG level (even more information)

setup_logging(verbose=0)


# EODAG - Classify

EODAG (Earth Observation Data Access Gateway) is a command line tool and a Python package for searching and downloading remotely sensed images while offering a unified API for data access regardless of the data provider.

EODAG gives you an easy way to access products from more than 10 providers, with more than 50 different product types (Sentinel 1, Sentinel 2, Sentinel 3, Landsat, etc.) that can be searched and downloaded.

In [ ]:
#Get Secrets from .env File
secrets = dotenv_values('.env')

In [ ]:
#Create Folders for saving Data, serializing and post processing.

# Path where the Data should be stored ('c:\\Users\\theUSER\\eodag-data')
root = '../eodag-data/'

workspace_download = os.path.join(root,'eodag_workspace_download')
workspace_serialize = os.path.join(root,'eodag_workspace_serialize_deserialize')
workspace_post_process = os.path.join(root,'eodag_workspace_post_process')
workspaces = [workspace_download, workspace_serialize, workspace_post_process]

for ws in workspaces:
    ws = os.path.abspath(ws)
    
    if not os.path.isdir(ws):
        os.mkdir(ws)
        print(f'Created Folder: {ws}')
    else:
        print(f'Folder already exists: {ws}')

## Step 1
### Configuration
In the configuration we pass the username and password from the Copernicus Dataspace Ecosystem (CDSE) to eodag. Also we define the path for the downloads.

In [ ]:
# 1. Configure
#Create EODAG Object and set preferred Provider
dag = EODataAccessGateway()
dag.set_preferred_provider("cop_dataspace") # Copernicus Data Space Ecosystem

dag.update_providers_config(f"""
    cop_dataspace:
        download:
            outputs_prefix: {os.path.abspath(workspace_download)}
        auth:
            credentials:
                username: {secrets['USER_KEY']}
                password: {secrets['USER_SECRET']}
""")


## Step 2
### Deserialize
Since the search is already done (see Notebook `eodag_search`) and the search result has been serialized, we are going to deserialize the search result and register it. If it is only deserialized it won't be able to download the data.

In [ ]:
# Deserialize the Search Results
output_file = os.path.join(workspace_serialize, "search_results3.geojson")
deserialized_search_results = dag.deserialize_and_register(output_file)

print(f"Got {len(deserialized_search_results)} deserialized products.")

In [ ]:
#Plot Quicklooks of Search Results
def plot_quicklooks(products):
    fig = plt.figure(figsize=(10,8))
    for i, product in enumerate(products[:12]):
        # This line takes care of downloading the quicklook
        quicklook_path = product.get_quicklook()
        
        date = product.properties['startTimeFromAscendingNode'][:16]
        provider = product.provider
        tile = product.properties['title'].split('_')[5].lstrip('T')
    
        # Plot the quicklook
        img = mpimg.imread(quicklook_path)
        ax = fig.add_subplot(3, 4, i+1)
        ax.set_title(f'Product {i}\n{date}\n{provider} - {tile}')
        ax.tick_params(top=False, bottom=False, left=False, right=False,
                       labelleft=False, labelbottom=False)
        plt.imshow(img)
    plt.tight_layout()
    
plot_quicklooks(deserialized_search_results)

## Step 3
### Download 
Now either a single product or multiple products from the search will be downloaded. If the product has already been downloaded it will not load it again, if it is saved in the right workingspace.

In [ ]:
# Download Single Product
product = deserialized_search_results[1]
path = dag.download(product)

In [ ]:
# Set Boundingbox for Area inside the Tile.
latmin, latmax = 48.1, 48.35
lonmin, lonmax = 16.1, 16.6
extent = {'lonmin': lonmin, 'latmin': latmin, 'lonmax': lonmax, 'latmax': latmax}

# Folium Map
fmap = folium.Map(location=(np.array([latmin, latmax]).mean(), np.array([lonmin, lonmax]).mean()), zoom_start=9)
folium.Rectangle(bounds=[[latmin, lonmin],[latmax, lonmax]], color="red").add_to(fmap)
folium.GeoJson(
    data=deserialized_search_results[:],  # SearchResult has a __geo_interface__ interface used by folium to get its GeoJSON representation, single results dont work (this [2:3] instead of [2])
    tooltip=folium.GeoJsonTooltip(fields=["title"])
).add_to(fmap)
fmap

In [ ]:
# Setting common Parameters for all further image processing
common_params = dict(
    crs=CRS.from_epsg(4326),               # the downloaded images are in 4326, don't reproject them
    resolution=0.0006,                     # but lower their resolution (0.0006 should be 60m in 100km)
    extent=(lonmin,latmin,lonmax,latmax)   # and zoom over/crop the area of interest
)

## Step 4 
### Post Process

#### Loading Bands as Dataset

The Level-2A processing includes a Scene Classification and an Atmospheric Correction applied to Top-Of-Atmosphere (TOA) Level-1C orthoimage products. Level-2A main output is an orthoimage atmospherically corrected, Surface Reflectance product.

Please be aware that "Surface Reflectance (SR)" is a new term that has been introduced to replace the former one: "Bottom of Atmosphere (BOA) reflectance."

Additional outputs are an Aerosol Optical Thickness (AOT) map, a Water Vapour (WV) map and a Scene Classification (SCL) map together with Quality Indicators (QI) for cloud and snow probabilities at 60 m resolution. Level-2A output image products are resampled and generated with an equal spatial resolution for all bands (10 m, 20 m or 60 m). Standard distributed products contain the envelope of all resolutions in three distinct folders:


- 10 m: containing spectral bands 2, 3, 4 , 8, a True Colour Image (TCI) and an AOT and WVP maps resampled from 20 m.

- 20 m: containing spectral bands 1 - 7, the bands 8A, 11 and 12, a True Colour Image (TCI), a Scene Classification (SCL) map and an AOT and WVP map. The band B8 is omitted as B8A provides more precise spectral information.

- 60 m: containing all components of the 20 m product resampled to 60 m and additionally the bands 1 and 9, a True Colour Image (TCI), a Scene Classification (SCL) map and an AOT and WVP map. The cirrus band 10 is omitted, as it does not contain surface information.

In [ ]:
# Get a list of all available Bands (assets)
def get_assets(root:str, res=60, only_spectral:bool=True, include_tci:bool=False):
    jp2_files = [file for dirs in os.walk(root, topdown=True)
                     for file in dirs[2] if file.endswith(f"_{res}m.jp2")]
    assets = [file.split('_')[2] for file in jp2_files if file.startswith('T')]

    if only_spectral and include_tci==False:
        assets = [a for a in assets if a[0]=='B']
    elif only_spectral and include_tci:
        assets = [a for a in assets if a[0] == 'B' or a[0] == 'T']
    else:
        pass
    
    return assets

In [ ]:
# Functions for loading data into datasets.

def load_single_product(product, bands:list):
    loaded_data = {}

    for band in bands:
        data = product.get_data(band=band, **common_params)
        data = data.squeeze()

        time_str = product.properties['startTimeFromAscendingNode']
        date = dt.datetime.strptime(time_str,'%Y-%m-%dT%H:%M:%S.%f%z')

        data = data.expand_dims(dim={'time':[date.date()]})
        data.name = band
        loaded_data[band] = data
    ds = xr.Dataset(loaded_data)
    return ds

def load_multiple_timestamps(products, bands:list):
    single_ds = []
    for product in products:
        single_product = load_single_product(product=product, bands=bands)
        single_ds.append(single_product)

    ds = xr.merge(single_ds)
    return ds

In [ ]:
assets = get_assets(path, res=10, only_spectral=True, include_tci=False)
assets

In [ ]:
for idx, elem in enumerate(assets):
    filepath = product.driver.get_data_address(product, band=elem).split('\\')[-5:]
    print(idx, filepath)

In [ ]:
# Loading multiple Bands into a dataset
ds = load_single_product(product=product, bands=assets)
ds

In [ ]:
single_img = ds.sel(time=dt.datetime(2023, 4, 22), method='nearest')

In [ ]:
def geojson_to_polygon(path:str) -> list:
    '''
    Reads a geojson File and returns a List of Polygons 

    Params:
        - ``path``: Filepath to Geojson

    Returns: 
        - ``polygons``: List of Polygons
    '''
    gdf = gpd.read_file(path)
    polygons = [feature for feature in gdf.geometry]
    return polygons

def geojson_to_polygon_dict(path:str) -> dict:
    '''
    Uses the Path of a Geojson File to extract the polygons and puts them into a Dictionary.

    Params: 
        - ``path``: Filepath to Geojson

    Returns: 
        - ``polygons_dict``
    '''
    polygons = geojson_to_polygon(path)
    polygons_dict = {idx: [polygon] for idx, polygon in enumerate(polygons)}
    return polygons_dict

def clip_array(ds:xr.Dataset, polygons):
    '''
    Takes an xarray.Dataset and a geometry and returns the xarray.Dataset, which has been spatialy clipped
    to the geometry.

    Params:
        - ``ds``: xarray.Dataset
        - ``polygons``: shapely.geometry
    
    Returns:
        - ``clipped_nan``: clipped dataset where values outside of polygons have Nan type
    '''
    clipped = ds.rio.clip(polygons, invert=False, all_touched=False, drop=True)
    clipped_nan = clipped.where(clipped == ds)
    return clipped_nan

def preprocess_data_to_classify(ds:xr.Dataset, feature_path:str, nonfeature_path:str, bands:list=None) -> list:
    '''
    Takes an xarray Dataset, two geojson files (one of areas with the desired feature, the other not with the feature)
    and a list of strings of the desired Bandnames in the Dataset and returns The Training and Test data for some Classifikators.

    Params:
        - ``ds``: xarray.Dataset
        - ``feature_path``: Filepath to Geojson with Polygons, which represent the Feature (e.g.: forested Areas)
        - ``nonfeature_path``: Filepath to Geojson, which does not have the feature (e.g.: not forested Areas)
        - ``bands`` (optional): List of Strings of desired Spectral Bands (e.g.: bands=['B02', 'B03', 'B04', 'B08'])
                                If None, then takes all in the Dataset.

    Returns:
        -  ``X_train, X_test, y_train, y_test``; Training and Test Split for scikit.learn Classificators
    '''
    # List all Bands which are loaded as Variables into the Dataset
    if bands == None:
        bands = list(ds.data_vars)

    # Geojsons from Features to Polygons
    polygons_feat = geojson_to_polygon_dict(feature_path)
    polygons_nonfeat = geojson_to_polygon_dict(nonfeature_path)

    # Dictionaries with Dataarrays, each clipped by a Polygon
    data_dict_feat = {idx: clip_array(ds, polygon) for idx, polygon in polygons_feat.items()}
    data_dict_nonfeat = {idx: clip_array(ds, polygon)  for idx, polygon in polygons_nonfeat.items()}

    # Median over time to get rid of outliers
    median_data_dict_feat = {idx: xarray.median(dim='time', skipna=True) for idx, xarray in data_dict_feat.items()}
    median_data_dict_nonfeat = {idx: xarray.median(dim='time', skipna=True) for idx, xarray in data_dict_nonfeat.items()}

    # Reshape the polygon dataarrays to get a tuple (one value per band) of pixel values
    feat_data = [xarray.to_array().values.reshape(len(bands),-1).T for xarray in median_data_dict_feat.values()]
    nonfeat_data = [xarray.to_array().values.reshape(len(bands),-1).T for xarray in median_data_dict_nonfeat.values()]

    # The rows of the different polygons are concatenated to a single array for further processing
    feat_values = np.concatenate(feat_data)
    nonfeat_values = np.concatenate(nonfeat_data)

    # Drop Nan Values
    X_feat_data = feat_values[~np.isnan(feat_values).any(axis=1)]
    X_nonfeat_data = nonfeat_values[~np.isnan(nonfeat_values).any(axis=1)]

    # Creating Output Vector (1 for pixel is features; 0 for pixel is not feature)
    y_feat_data = np.ones(X_feat_data.shape[0])
    y_nonfeat_data = np.zeros(X_nonfeat_data.shape[0])
    
    # Concatnate all Classes for training 
    X = np.concatenate([X_feat_data, X_nonfeat_data])
    y = np.concatenate([y_feat_data, y_nonfeat_data])

    # Split into Training and Testing Data.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

    return X_train, X_test, y_train, y_test

## Classify

### Preparing the data

In [ ]:
path_forest = r'..\eodag-data\shpfiles\forest.geojson'
path_nonforest = r'..\eodag-data\shpfiles\nonforest.geojson'

In [ ]:
X_train, X_test, y_train, y_test = preprocess_data_to_classify(ds=ds, feature_path=path_forest, nonfeature_path=path_nonforest)

In [ ]:
# Naive Bayes
nb = GaussianNB()
nb_test = nb.fit(X_train, y_train)
nb_predict = nb.predict(X_test)


In [ ]:
print("NAIVE BAYES: \n "+ classification_report(y_test, nb_predict))

In [ ]:
print("NAIVE BAYES: \n ",confusion_matrix(y_test, nb_predict), "\n")


### Applying a classifier to an image

In [ ]:
bands = []
for band in ['B04', 'B03', 'B02', 'B08']:
    bands.append(single_img[band].values)
    
image_data = np.stack(bands, axis=2)

In [ ]:
num_of_pixels = single_img.sizes['x'] * single_img.sizes['y']
num_of_bands = len(bands)
X_image_data = image_data.reshape(num_of_pixels, num_of_bands)

nb_predict_img = nb.predict(X_image_data)

nb_predict_img = nb_predict_img.reshape(single_img.sizes['y'], single_img.sizes['x'])

In [ ]:
cmap = colors.ListedColormap([(1, 0, 0, 0), 'g'])

fig, ax = plt.subplots(figsize=(12,3))
ax.imshow(nb_predict_img, cmap=cmap)
ax.set_title("naive Bayes")
ax.set_axis_off()
plt.show()

In [ ]:
# Adding the Classification Array to the Dataset. (An already existing band gets "copied" and its values get overwritten; there might be better ways, but this is short)
ds['NB-forest'] = ds['B02']
ds['NB-forest'].values = np.array([nb_predict_img])
ds

In [ ]:
ds.sel(time=dt.datetime(2023, 4, 22), method='nearest')['NB-forest'].plot.imshow()

In [ ]:
# Loading the True Color Image
tci = load_single_product(product=product, bands=['TCI'])
tci = tci.sel(time=dt.datetime(2023, 4, 22), method='nearest')

In [ ]:
# Plotting Forest Mask on top of TCI
cmap = colors.ListedColormap([(1, 0, 0, 0), 'C0'])

fig, ax = plt.subplots(figsize=(8,8))
tci['TCI'].plot.imshow(ax=ax, zorder=0)
ds.sel(time=dt.datetime(2023, 4, 22), method='nearest')['NB-forest'].plot.imshow(ax=ax, zorder=1, cmap=cmap, alpha=1)
plt.show()

### NDVI

NDVI and many other indices rely on the normalized difference, represented by the function below

In [ ]:
def normalized_difference(a, b):
    return (a - b*1.)/(a + b) # If b in numerator is not multiplied by 1 as a float some weird things happen.

To get the ndvi we need to calculate the normalized difference between the infrared (B08) and the red (B04) band.

In [ ]:
# Calculating the NDVI and adding it to the dataset
ndvi = normalized_difference(single_img['B08'], single_img['B04'])
single_img['NDVI'] = ndvi

We can plot the result:

In [ ]:
single_img['NDVI'].plot.imshow()

NDVI does a good job seperating vegetation from non-vegetation but it can't seperate forest from vegetated cropland, or grassland.

In [ ]:
# Defining a function which adds any Index to the dataset might helps keeping your code clean.
def add_index_to_dataset(data:xr.Dataset, index:str) -> xr.Dataset:
    '''
    Takes an xarray Dataset and calculates and adds one of the indices:
    
    Params:
        - ``data``: xarray.Dataset
        - ``index``: string (one of [``NDVI``, ``NDMI``, ``MSI``, ``NDWI``])

    Returns:
        - ``data`` with Index as new Variable
    '''
    indices = {'NDVI':{'name':'Normalized Diffference Vegetation Index',
                       'function':normalized_difference(data['B08'], data['B04'])},
            #    'NDMI': {'name':'Normalized Diffference Moisture Index',
            #            'function':normalized_difference(data['B08'], data['B11'])},
            #    'MSI': {'name':'Moisture Stress Index',
            #            'function':data['B11'] / data['B08']},
               'NDWI': {'name':'Normalized Diffference Water Index',
                       'function':normalized_difference(data['B03'], data['B08'])},}
    data[index] = indices[index]['function']
    print(f'{indices[index]['name']} ({index}) has been calculated.')
    return data

In [ ]:
ds = add_index_to_dataset(ds, index='NDVI')
ds = add_index_to_dataset(ds, index='NDWI')

In [ ]:
ds['NDWI'].sel(time=dt.datetime(2021, 4, 12), method='nearest').plot.imshow()